**To Make sure the code works, install these packages** 
- pip install beautifulsoup4
- python -m pip install requests
- pip install xlsxwriter
- pip uninstall selenium
- pip install selenium==4.9.0 { https://medium.com/@ayanchowdhury00/web-scrapping-on-firefox-browser-using-selenium-in-python-edcf678a1dea}
- pip install transformers

*Importing needed modules*

In [1]:
import requests
#import re
from bs4 import BeautifulSoup as bs
import time
import selenium
#print(selenium.__version__)
from selenium import webdriver
import xlsxwriter
import pandas as pd
from transformers import pipeline
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# setting environment variable to disable tokenizer parallelism
os.environ["TOKENIZERS_PARALLELISM"] = "false"



In [2]:
# using firefox driver to load the url and disable cookies! using depreciated selenium version for using firefox profile
options=webdriver.FirefoxOptions()
### to disable cookies 
fp = webdriver.FirefoxProfile()
fp.set_preference("network.cookie.cookieBehavior",2);
driver = webdriver.Firefox(firefox_profile=fp)
base_url = "https://www.sanofi.com"
url = base_url+"/en/media-room/press-releases"
driver.get(url)
time.sleep(10) 




/var/folders/48/ks0jnbtd2yd8b1nhh_z_t7340000gn/T/ipykernel_47074/2426025562.py:4: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  fp = webdriver.FirefoxProfile()
/var/folders/48/ks0jnbtd2yd8b1nhh_z_t7340000gn/T/ipykernel_47074/2426025562.py:6: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  driver = webdriver.Firefox(firefox_profile=fp)


In [3]:
## defining function to keep abstract of maximum 512 words, which is used to summarize the abstract using bart-large-cnn model
def truncate_by_words(text, max_words=512):
    words = text.split()
    return ' '.join(words[:max_words])
    
model_name="facebook/bart-large-cnn"
summarizer = pipeline("summarization", model=model_name, tokenizer=model_name)

In [4]:
# defining lists to extract this info from the url
Date = []
Title = []
URL = []
PDF_URL = []
Abstract = []
Summaries = []

In [5]:
## this is the main function that does extracts all needed info for us!

def find_page(path):
 try:
    ## finiding if exists a next button on the current page 
    button = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, path)))
    #button = driver.find_element(By.XPATH, path)
 except:
    print("page not found")
    return False
 else:
    global i
    print("page exists", i)
    soup=bs(driver.page_source) 
    body = soup.find_all('div', class_='MuiBox-root css-y36fim')
    for article in body:
        date = article.find('div', class_="MuiTypography-root MuiTypography-body2 css-1ufdk6m-MuiTypography-root")
        title = article.find('div', class_="MuiTypography-root MuiTypography-h3 css-1fchgxt-MuiTypography-root")
        link = article.find('a', class_="elements-ds-dgpxh5")
        print(date.get_text(),"\n",title.get_text(),"\n",base_url+link.get("href"))
        
        Date.append(date.get_text())
        Title.append(title.get_text())
        URL.append(base_url+link.get("href"))
    
        #### Getting details of news_article   
         
        detail_article = requests.get(base_url+link.get("href"))  
        detail_soup = bs(detail_article.content, 'html.parser')
        pdflink = detail_soup.find('a', class_="elements-ds-twdzwl").get("href")
        PDF_URL.append(base_url+pdflink)
        abstract = detail_soup.find_all('div', class_="MuiGrid2-root MuiGrid2-direction-xs-row MuiGrid2-grid-mobile-4 MuiGrid2-grid-tablet-6 MuiGrid2-grid-desktop-8 css-1blu0l0-MuiGrid2-root")
        Abstract_detail = []
        Summary = []
        for detail in abstract:
            #print(detail.get_text(),"\n")
            Abstract_detail.append(detail.get_text())
            if len(detail.get_text())>250:  
                try:
                    # Truncate abstract to fit within a word limit
                    truncated_abstract = truncate_by_words(detail.get_text(), max_words=500)
                    # Generate the summary using summarizer pipeline
                    summary = summarizer(truncated_abstract, max_length=100, min_length=25, do_sample=False)
                    
                    if summary and 'summary_text' in summary[0]:
                        Summary.append(summary[0]['summary_text'])
                    else:
                        Summary.append("Summary not available")
                except Exception as e:
                    Summary.append(f"Error: {str(e)}")
            else:
                Summary.append(detail.get_text())
        
        Abstract.append(Abstract_detail)
        Summaries.append(Summary)
        
    button.click()    
    time.sleep(2) 
    i=i+1
    return True



In [6]:
## xpath of next button on first page copied from inspecting source - can change depending on when this code is tried 
## and if meanwhile something has changed on the website
button = "/html/body/div/main/div[3]/div[1]/div[1]/div/div[22]/nav/ul/li[10]/button"

# defining counter to go on different pages of press-releases
i=1

# using recursive function to go to the required page and until the last page to extract all information
# remember i = page number so can change this piece of code depending on till when you want to extract the information
while find_page(button) and i < 2:
    find_page(button)
else: 
    # going to the last page here as on last page, next page button doesn't work so it complains, also update xpath if it changes for 
    #the last page button
    last_page = "/html/body/div/main/div[3]/div[1]/div[1]/div/div[17]/nav/ul/li[9]/button"
    find_page(last_page)


page exists 1
September 2, 2024 
 Press Release: Tolebrutinib meets primary endpoint in HERCULES phase 3 study, the first and only to show reduction in disability accumulation in non-relapsing secondary progressive multiple sclerosis 
 https://www.sanofi.com/en/media-room/press-releases/2024/2024-09-02-05-00-00-2938875
August 26, 2024 
 Media Update: ERS: New data highlight Sanofi’s scientific innovation and leadership in immune-mediated respiratory diseases 
 https://www.sanofi.com/en/media-room/press-releases/2024/2024-08-26-05-00-00-2935223
August 8, 2024 
 Press Release: Sarclisa induction treatment demonstrated significantly improved progression-free survival in patients with newly diagnosed multiple myeloma eligible for transplant 
 https://www.sanofi.com/en/media-room/press-releases/2024/2024-08-08-05-00-00-2926566
July 25, 2024 
 Press Release: Online availability of Sanofi’s half-year financial report for 2024 
 https://www.sanofi.com/en/media-room/press-releases/2024/2024-07-

In [7]:
# defining the data frame to be exported to excel file with all the information
df = pd.DataFrame(list(zip(Date, Title, URL, PDF_URL, Abstract, Summaries)), columns=['Date','Title', 'URL','PDF_URL','Abstract', 'Summary'])

In [8]:
# write DataFrame to excel
datatoexcel = pd.ExcelWriter('Sanofi.xlsx')
df.to_excel(datatoexcel) 
# save the excel
datatoexcel.close()
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.
